In [2]:
using CSV
using DataFrames
using Random
using MLJ
using DecisionTree
using PyPlot
using Statistics

Cargamos los datos de viviendas usadas

In [7]:
data = CSV.File("data.csv") |> DataFrame

Row,Column1,X,Precio,Superficie_Terreno,Superficie_Habitacional,Agno_Construccion,Antiguedad_Promedio,Avaluo_Fiscal,Avaluo_Fiscal_Unitario,Barrio,Calidad_Habitacional,IPV_General,Latitud,Longitud,Tipo_Comprador_Simple,Tipo_Vendedor_Simple,Trimestre,Valor_Unitario_Terreno,Valor_Unitario_Construccion_Habitacional_Promedio,Fecha_Transaccion,Precio_Unitario
,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,String31,Float64,Float64,Float64,Float64,String7,String7,String3,Float64,Float64,Date,Float64
1,1,1,8400,180,88,1953,67.0,4733.21,26.2956,parque_bicentenario,3.0,206.73,-33.4044,-70.5955,Persona,Persona,T4,18.8677,15.1877,2020-12-16,46.6667
2,2,2,27300,300,114,1953,68.0,7625.95,25.4198,parque_bicentenario,3.0,219.441,-33.4031,-70.5942,Empresa,Persona,T3,18.8677,15.1877,2021-09-08,91.0
3,3,3,14800,230,135,1957,52.3333,6475.01,28.1522,parque_bicentenario,3.0,193.66,-33.4046,-70.5904,Empresa,Persona,T4,18.8677,15.1877,2018-10-28,64.3478
4,4,4,14800,230,135,1957,54.3333,6475.01,28.1522,parque_bicentenario,3.0,206.73,-33.4046,-70.5904,Empresa,Empresa,T4,18.8677,15.1877,2020-10-30,64.3478
5,5,5,19000,416,242,1957,39.7411,13045.9,31.3605,parque_bicentenario,1.86777,163.823,-33.4027,-70.5891,Persona,Persona,T3,19.3633,20.6557,2016-08-26,45.6731
6,6,6,8326,432,130,1958,57.0,10543.8,24.407,parque_bicentenario,4.0,163.714,-33.4022,-70.5892,Persona,Empresa,T3,19.3633,8.38841,2015-09-11,19.2731
7,7,7,24820,270,239,1958,56.3969,8417.52,31.176,parque_bicentenario,3.0,204.937,-33.4021,-70.5892,Empresa,Persona,T1,19.3633,15.0891,2020-02-22,91.9259
8,8,8,11000,731,190,1957,62.0,18246.7,24.9612,parque_bicentenario,3.0,203.461,-33.4021,-70.5889,Empresa,Persona,T4,19.3633,15.1877,2019-12-01,15.0479
9,9,9,25000,731,190,1957,62.0,18246.7,24.9612,parque_bicentenario,3.0,203.461,-33.4021,-70.5889,Empresa,Persona,T4,19.3633,15.1877,2019-12-01,34.1997


Se agrega columna Proporcion a data_brut y se filtran los datos para los cuales el Precio es menor al Avaluo_Fiscal

In [8]:
data.Proporcion = data.Precio ./ data.Avaluo_Fiscal;
data = filter(row -> row.Proporcion >= 1, data);

Se agregan columnas adicionales necesarias para ajustar modelo predictivo 

In [10]:
data.log_Precio = log.(data.Precio);
data.raiz_Superficie_Terreno = sqrt.(data.Superficie_Terreno);
data.raiz_Superficie_Habitacional = sqrt.(data.Superficie_Habitacional);
data.log_Avaluo_Fiscal = log.(data.Avaluo_Fiscal);
data.log_Avaluo_Fiscal_Unitario = log.(data.Avaluo_Fiscal_Unitario);
data.log_Valor_Unitario_Terreno = log.(data.Valor_Unitario_Terreno);
data.log_Valor_Unit_Const_Hab = log.(data.Valor_Unitario_Construccion_Habitacional_Promedio);


Se establecen las features que se utilizarán en el modelo predictivo

In [11]:
features = [
    :log_Precio,
    :raiz_Superficie_Terreno,
    :raiz_Superficie_Habitacional,
    :Agno_Construccion,
    :Antiguedad_Promedio,
    :log_Avaluo_Fiscal,
    :log_Avaluo_Fiscal_Unitario,
    :Barrio,
    :Calidad_Habitacional,
    :IPV_General,
    :Latitud,
    :Longitud,
    :Tipo_Comprador_Simple,
    :Tipo_Vendedor_Simple,
    :Trimestre,
    :log_Valor_Unitario_Terreno,
    :log_Valor_Unit_Const_Hab
]
data = data[:, features]
names(data)

17-element Vector{String}:
 "log_Precio"
 "raiz_Superficie_Terreno"
 "raiz_Superficie_Habitacional"
 "Agno_Construccion"
 "Antiguedad_Promedio"
 "log_Avaluo_Fiscal"
 "log_Avaluo_Fiscal_Unitario"
 "Barrio"
 "Calidad_Habitacional"
 "IPV_General"
 "Latitud"
 "Longitud"
 "Tipo_Comprador_Simple"
 "Tipo_Vendedor_Simple"
 "Trimestre"
 "log_Valor_Unitario_Terreno"
 "log_Valor_Unit_Const_Hab"

Se ajustan los tipos de de algunas features 

In [13]:
coerce!(data, :Precio => Continuous, :Superficie_Terreno => Continuous, :Superficie_Habitacional => Continuous);
coerce!(data, :Barrio => Multiclass, :Tipo_Comprador_Simple => Multiclass, :Tipo_Vendedor_Simple => Multiclass, :Trimestre => Multiclass);

Se modifican las variables Multiclass por oneHot

In [17]:
hot = OneHotEncoder(drop_last=false);
mach = MLJ.fit!(machine(hot, data));
data_oneHot = MLJ.transform(mach, data);

┌ Info: Training machine(OneHotEncoder(features = Symbol[], …), …).
└ @ MLJBase C:\Users\rjvia\.julia\packages\MLJBase\fEiP2\src\machines.jl:492
┌ Info: Spawning 11 sub-features to one-hot encode feature :Barrio.
└ @ MLJModels C:\Users\rjvia\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878
┌ Info: Spawning 2 sub-features to one-hot encode feature :Tipo_Comprador_Simple.
└ @ MLJModels C:\Users\rjvia\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878
┌ Info: Spawning 2 sub-features to one-hot encode feature :Tipo_Vendedor_Simple.
└ @ MLJModels C:\Users\rjvia\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878
┌ Info: Spawning 4 sub-features to one-hot encode feature :Trimestre.
└ @ MLJModels C:\Users\rjvia\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878


Se separan los datos en y, X y luego se particionan en train (80%) y test (20%)

In [16]:
y, X = unpack(data_oneHot, ==(:log_Precio))
pos_train, pos_test = partition(collect(eachindex(y)), 0.8, shuffle=true, rng=5)

([1394, 1398, 922, 294, 1509, 1099, 249, 88, 1146, 2334  …  250, 2295, 98, 1502, 2362, 1052, 62, 1364, 749, 1605], [1390, 743, 604, 1287, 547, 445, 2319, 630, 2021, 970  …  1679, 1040, 2113, 485, 2284, 1668, 2358, 66, 7, 997])